In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

transformers.logging.set_verbosity_error()
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

llm = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it", dtype=dtype, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
v.torch.load("steering_vectors.pt", map_location=device)[(15, 21576)].to(device, dtype=dtype)

layer_idx = 15
coefficient = 4.0

def steering_hook(module, input, output):
    return output + coefficient * v

# register
hook_handle = llm.model.layers[layer_idx].mlp.register_forward_hook(steering_hook)

In [ ]:
messages = [{"role": "user", "content": "Hello, how are you?"}]
input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True
).to(device)

output_ids = llm.generate(**input_ids, max_new_tokens=128, do_sample=False, repetition_penalty=1.1)
answer = output_ids.tolist()[0][input_ids.input_ids.shape[1]:]
print(tokenizer.decode(answer, skip_special_tokens=True))